# Machine Learning Engineer Nanodegree
## Unsupervised Learning
## Project 3: Creating Customer Segments

Welcome to the third project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

## Getting Started

In this project, you will analyze a dataset containing data on various customers' annual spending amounts (reported in *monetary units*) of diverse product categories for internal structure. One goal of this project is to best describe the variation in the different types of customers that a wholesale distributor interacts with. Doing so would equip the distributor with insight into how to best structure their delivery service to meet the needs of each customer.

The dataset for this project can be found on the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Wholesale+customers). For the purposes of this project, the features `'Channel'` and `'Region'` will be excluded in the analysis — with focus instead on the six product categories recorded for customers.

Run the code block below to load the wholesale customers dataset, along with a few of the necessary Python libraries required for this project. You will know the dataset loaded successfully if the size of the dataset is reported.

In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import renders as rs
from IPython.display import display # Allows the use of display() for DataFrames

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Load the wholesale customers dataset
try:
    data = pd.read_csv("customers.csv")
    data.drop(['Region', 'Channel'], axis = 1, inplace = True)
    print "Wholesale customers dataset has {} samples with {} features each.".format(*data.shape)
except:
    print "Dataset could not be loaded. Is the dataset missing?"

Wholesale customers dataset has 440 samples with 6 features each.


## Data Exploration
In this section, you will begin exploring the data through visualizations and code to understand how each feature is related to the others. You will observe a statistical description of the dataset, consider the relevance of each feature, and select a few sample data points from the dataset which you will track through the course of this project.

Run the code block below to observe a statistical description of the dataset. Note that the dataset is composed of six important product categories: **'Fresh'**, **'Milk'**, **'Grocery'**, **'Frozen'**, **'Detergents_Paper'**, and **'Delicatessen'**. Consider what each category represents in terms of products you could purchase.

In [2]:
# Display a description of the dataset
display(data.describe())

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455
std,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937
min,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000
25%,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000
50%,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000
75%,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000
max,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000


### Implementation: Selecting Samples
To get a better understanding of the customers and how their data will transform through the analysis, it would be best to select a few sample data points and explore them in more detail. In the code block below, add **three** indices of your choice to the `indices` list which will represent the customers to track. It is suggested to try different sets of samples until you obtain customers that vary significantly from one another.

In [3]:
# TODO: Select three indices of your choice you wish to sample from the dataset
indices = [0, 220, 439]

# Create a DataFrame of the chosen samples
samples = pd.DataFrame(data.loc[indices], columns = data.keys()).reset_index(drop = True)
print "Chosen samples of wholesale customers dataset:"
display(samples)


Chosen samples of wholesale customers dataset:


,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
0,12669,9656,7561,214,2674,1338
1,14755,899,1382,1765,56,749
2,2787,1698,2510,65,477,52


### Question 1
Consider the total purchase cost of each product category and the statistical description of the dataset above for your sample customers.  
*What kind of establishment (customer) could each of the three samples you've chosen represent?*  
**Hint:** Examples of establishments include places like markets, cafes, and retailers, among many others. Avoid using names for establishments, such as saying *"McDonalds"* when describing a sample customer as a restaurant.

In [4]:
# A bar plot to visualise the amount of each product purchased for each sample, together with the dataset mean
import seaborn as sns
samples_bar = samples.append(data.describe().loc['mean'])
samples_bar.index = indices + ['mean']
my_bar = samples_bar.plot(kind='bar', figsize=(14,6)) 

ImportError: No module named seaborn

In [ ]:
# a simple heat map of the percentiles from dataset for comparing percentiles can also be more helpful 
#calculate percentile ranks of the data
percentiles = data.rank(pct=True)

# Round it up and multiply by 100
percentiles = 100*percentiles.round(decimals=3)

# Select indices I chose from the percentiles dataFrame
percentiles = percentiles.iloc[indices]

# create the heat map using the seaborn library
my_map = sns.heatmap(percentiles, vmin=1, vmax=99, annot=True)


**Answer:**

 Comparing each feature amounts to the mean values from the above graph and heat map of the percentiles, it is necessary to notice that the first sample point or sample point 0 purchased an amount of 12,669 of Fresh, about 668.70228 above the mean value of 12000.29772. Additionally, Sample point 0 purchased about (9656 of Milk for sample point 0 - 5796.265909 of Milk for the mean) 3859.734091 Milk over the mean; and nearly bellow the mean for  Grocery feature about (7561 of Grocery for sample point 0 out of 7951.277273 of mean for Grocery) 30.277273 bellow the mean. Sample 0 is about 2857.931818, 207.483182, and 186.870455 of Frozen, Detergents_Paper and Delicatessen respectively bellow the mean. Sample point 0 has high spending in relation to the category mean for all categories except Frozen, it could be a big box retailer because most consumers like myself buy their Grocery, Milk, Fresh and Detergents from retail stores therefore they are more likely to purchase more of these products. 
 Sample point 1 has a significant number of Fresh(70 percentiles) and miningful number of Frozen(55 percentiles) and Delicatessen(42 percentiles) and few amounts of Milk(11 percentiles), Grocery(10 percentiles) and Detergents_Paper(4.7 percentiles). It could be a hotel because hotels are more likely to sell Fresh, Frozen and Delicatessen than Grocery and Detergents_Paper.
 Sample point 2 has a good amount of Detergents_Paper(40 percentiles) and a few number of Grocery(31 percentiles) and Milk(27 percentiles) and few amounts of Fresh(22 percentiles), Delicatessen(3.9 percentiles) and Frozen(2 percentiles). It could be a restaurant because restaurant are more likely to sell food products therefore purchase Grocery, Milk and Fresh. 
 
 


### Implementation: Feature Relevance
One interesting thought to consider is if one (or more) of the six product categories is actually relevant for understanding customer purchasing. That is to say, is it possible to determine whether customers purchasing some amount of one category of products will necessarily purchase some proportional amount of another category of products? We can make this determination quite easily by training a supervised regression learner on a subset of the data with one feature removed, and then score how well that model can predict the removed feature.

In the code block below, you will need to implement the following:
 - Assign `new_data` a copy of the data by removing a feature of your choice using the `DataFrame.drop` function.
 - Use `sklearn.cross_validation.train_test_split` to split the dataset into training and testing sets.
   - Use the removed feature as your target label. Set a `test_size` of `0.25` and set a `random_state`.
 - Import a decision tree regressor, set a `random_state`, and fit the learner to the training data.
 - Report the prediction score of the testing set using the regressor's `score` function.

In [ ]:
# TODO: Make a copy of the DataFrame, using the 'drop' function to drop the given feature
feature = 'Delicatessen'
new_data = data.drop(feature, axis = 1)

# TODO: Split the data into training and testing sets using the given feature as the target
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new_data.values, data[feature], test_size=0.25, random_state=0)

# TODO: Create a decision tree regressor and fit it to the training set
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

regressor = DecisionTreeRegressor(random_state=42) 
regressor = regressor.fit(X_train, y_train)

# TODO: Report the score of the prediction using the testing set

score = regressor.score(X_test, y_test)
 
print "score: " , score

### Question 2
*Which feature did you attempt to predict? What was the reported prediction score? Is this feature is necessary for identifying customers' spending habits?*  
**Hint:** The coefficient of determination, `R^2`, is scored between 0 and 1, with 1 being a perfect fit. A negative `R^2` implies the model fails to fit the data.

**Answer:**
I attempted to predict 'Delicatessen'.
The reported prediction score was -11.6636871594.
This feature is necessary for identifying customers' spending habits because a negative R^2 score means that the feature is relevant, if we were to remove this feature we would lose significant amount of customers' spending information.  


### Visualize Feature Distributions
To get a better understanding of the dataset, we can construct a scatter matrix of each of the six product features present in the data. If you found that the feature you attempted to predict above is relevant for identifying a specific customer, then the scatter matrix below may not show any correlation between that feature and the others. Conversely, if you believe that feature is not relevant for identifying a specific customer, the scatter matrix might show a correlation between that feature and another feature in the data. Run the code block below to produce a scatter matrix.

In [ ]:
# Produce a scatter matrix for each pair of features in the data
pd.scatter_matrix(data, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

### Question 3
*Are there any pairs of features which exhibit some degree of correlation? Does this confirm or deny your suspicions about the relevance of the feature you attempted to predict? How is the data for those features distributed?*  
**Hint:** Is the data normally distributed? Where do most of the data points lie? 

In [ ]:
import matplotlib.pyplot as plt
# it is also interesting to look at:
#the scatter matrix with the features reordered with highly correlated features first (correlation values are in red text)
log_data = np.log(data)
idx_reorder = ['Detergents_Paper', 'Grocery', 'Milk', 'Fresh', 'Frozen', 'Delicatessen']
axes = pd.scatter_matrix(log_data[idx_reorder], alpha = 0.3, figsize = (14,8), diagonal = 'kde')
corr = log_data[idx_reorder].corr().as_matrix()

for i, j in zip(*plt.np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%+.3f" %corr[i,j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center',color="red",
                        fontsize=14)

**Answer:**
From the above plot we can see that Detergents_Paper and Grocery are highly correlated with coefficient of correlation close to 1 and neither of them is considerably correlated to Delicatessen feature I was trying to predict. for example if I were to figure out a data feature relevance on a real estate housing data where the features include the the length, the width, the perimeter, the area of a land, the number of rooms and the number of murder or security level of the given housing area . It was going to be obvious that the perimeter and the area of the land marks were almost 100% correlated to the length and the width of each land marks, because length by width gives the area, also 2 by Length plus width gives the perimeter, therefore, I could remove either the length and width or the perimeter and the area to get less features, but neither of both was going to be directly correlated with the number of murder or security level around the given housing area, because the security level feature cannot be known from length and width, therefore it is relevant like Delicatessen feature is relevant for the customer segement data.   
The Data features are positively skewed distribution, most of the data points lie to left of the graph towards zero.
As you can see from the above scatterplot the correlation values show that Detergents_Paper, Grocery are exibiting high amount of correlation. 


## Data Preprocessing
In this section, you will preprocess the data to create a better representation of customers by performing a scaling on the data and detecting (and optionally removing) outliers. Preprocessing data is often times a critical step in assuring that results you obtain from your analysis are significant and meaningful.

### Implementation: Feature Scaling
If data is not normally distributed, especially if the mean and median vary significantly (indicating a large skew), it is most [often appropriate](http://econbrowser.com/archives/2014/02/use-of-logarithms-in-economics) to apply a non-linear scaling — particularly for financial data. One way to achieve this scaling is by using a [Box-Cox test](http://scipy.github.io/devdocs/generated/scipy.stats.boxcox.html), which calculates the best power transformation of the data that reduces skewness. A simpler approach which can work in most cases would be applying the natural logarithm.

In the code block below, you will need to implement the following:
 - Assign a copy of the data to `log_data` after applying a logarithm scaling. Use the `np.log` function for this.
 - Assign a copy of the sample data to `log_samples` after applying a logrithm scaling. Again, use `np.log`.

In [ ]:
# TODO: Scale the data using the natural logarithm
log_data = np.log(data)

# TODO: Scale the sample data using the natural logarithm
log_samples = np.log(samples)

# Produce a scatter matrix for each pair of newly-transformed features
pd.scatter_matrix(log_data, alpha = 0.3, figsize = (14,8), diagonal = 'kde');



### Observation
After applying a natural logarithm scaling to the data, the distribution of each feature should appear much more normal. For any pairs of features you may have identified earlier as being correlated, observe here whether that correlation is still present (and whether it is now stronger or weaker than before).

Run the code below to see how the sample data has changed after having the natural logarithm applied to it.

In [ ]:
# Display the log-transformed sample data
display(log_samples)

### Implementation: Outlier Detection
Detecting outliers in the data is extremely important in the data preprocessing step of any analysis. The presence of outliers can often skew results which take into consideration these data points. There are many "rules of thumb" for what constitutes an outlier in a dataset. Here, we will use [Tukey's Method for identfying outliers](http://datapigtechnologies.com/blog/index.php/highlighting-outliers-in-your-data-with-the-tukey-method/): An *outlier step* is calculated as 1.5 times the interquartile range (IQR). A data point with a feature that is beyond an outlier step outside of the IQR for that feature is considered abnormal.

In the code block below, you will need to implement the following:
 - Assign the value of the 25th percentile for the given feature to `Q1`. Use `np.percentile` for this.
 - Assign the value of the 75th percentile for the given feature to `Q3`. Again, use `np.percentile`.
 - Assign the calculation of an outlier step for the given feature to `step`.
 - Optionally remove data points from the dataset by adding indices to the `outliers` list.

**NOTE:** If you choose to remove any outliers, ensure that the sample data does not contain any of these points!  
Once you have performed this implementation, the dataset will be stored in the variable `good_data`.

In [ ]:
# Create an array of all outliers
all_outliers = np.array([], dtype='int64')

# For each feature find the data points with extreme high or low values
for feature in log_data.keys():

    # TODO: Calculate Q1 (25th percentile of the data) for the given feature
    Q1 = np.percentile(log_data[feature], 25)

    # TODO: Calculate Q3 (75th percentile of the data) for the given feature
    Q3 = np.percentile(log_data[feature], 75)

    # TODO: Use the interquartile range to calculate an outlier step (1.5 times the interquartile range)
    step = 1.5*(Q3 - Q1)

    # Catch the outliers
    outlier_points = log_data[~((log_data[feature] >= Q1 - step) & (log_data[feature] <= Q3 + step))]
    all_outliers = np.append(all_outliers, outlier_points.index.values.astype('int64'))
    #print all_outliers

    # Display the outliers
    print "Data points considered outliers for the feature '{}':".format(feature)
    display(outlier_points)

# OPTIONAL: Select the indices for data points you wish to remove
outliers  = []

# Count the unique elements in the all_outliers array
all_outlier, indices = np.unique(all_outliers, return_inverse=True)
counts = np.bincount(indices)

# Obtain outliers using the counts
outliers = all_outlier[counts>1]
print outliers

# Remove the outliers, if any were specified
good_data = log_data.drop(log_data.index[outliers]).reset_index(drop = True)

### Question 4
*Are there any data points considered outliers for more than one feature? Should these data points be removed from the dataset? If any data points were added to the `outliers` list to be removed, explain why.* 

**Answer:**
Yes, there are many data points considered outliers for more than one feature.
Yes, these data points should be removed from the dataset, because they can negatively affect the mean, the standard deviation. Some of the data points are very high and other are very smaller than the mid range value to the point that they can lead to many statistical miscalculation and may also result in wrong data analysis. 

## Feature Transformation
In this section you will use principal component analysis (PCA) to draw conclusions about the underlying structure of the wholesale customer data. Since using PCA on a dataset calculates the dimensions which best maximize variance, we will find which compound combinations of features best describe customers.

### Implementation: PCA

Now that the data has been scaled to a more normal distribution and has had any necessary outliers removed, we can now apply PCA to the `good_data` to discover which dimensions about the data best maximize the variance of features involved. In addition to finding these dimensions, PCA will also report the *explained variance ratio* of each dimension — how much variance within the data is explained by that dimension alone. Note that a component (dimension) from PCA can be considered a new "feature" of the space, however it is a composition of the original features present in the data.

In the code block below, you will need to implement the following:
 - Import `sklearn.decomposition.PCA` and assign the results of fitting PCA in six dimensions with `good_data` to `pca`.
 - Apply a PCA transformation of the sample log-data `log_samples` using `pca.transform`, and assign the results to `pca_samples`.

In [ ]:
# TODO: Apply PCA to the good data with the same number of dimensions as features
from sklearn.decomposition import PCA

pca = PCA(n_components = None).fit(good_data)

# TODO: Apply a PCA transformation to the sample log-data
pca_samples = pca.transform(log_samples)

# Generate PCA results plot
pca_results = rs.pca_results(good_data, pca)

print pca_results['Explained Variance'].cumsum()

### Question 5
*How much variance in the data is explained* ***in total*** *by the first and second principal component? What about the first four principal components? Using the visualization provided above, discuss what the first four dimensions best represent in terms of customer spending.*  
**Hint:** A positive increase in a specific dimension corresponds with an *increase* of the *positive-weighted* features and a *decrease* of the *negative-weighted* features. The rate of increase or decrease is based on the indivdual feature weights.

**Answer:**
For the first principal component, we have 0.4426 explained variance and 0.4430 total variance, a significant positive weight is placed on Detergents_Paper with meaningful positive weight on Milk and Grocery. This dimension is best categorized by customer spending on retail goods. for the second pricipal component, we have 0.2761 Explained variance and 0.7068 total variance, a significant positive weight is placed on Fresh with meaningful positive weight on Frozen and Delicatessen. This dimension is best categorized by customer spending on restaurants and cafe goods.
The third pricipal components, with total variance of 0.8299, a significant positive weight is placed on Delicatessen with meaningful positive weight on Frozen. This dimension is best categorized by customer spending on Hotel and restaurant goods.
The fourth principal component, 0.9311 total variance, with significant positive weight is placed on Frozen with meaningful positive weight on Detergents_Paper. This dimension is best categorized by customer spending on retail goods.

### Observation
Run the code below to see how the log-transformed sample data has changed after having a PCA transformation applied to it in six dimensions. Observe the numerical value for the first four dimensions of the sample points. Consider if this is consistent with your initial interpretation of the sample points.

In [ ]:
# Display sample log-data after having a PCA transformation applied
display(pd.DataFrame(np.round(pca_samples, 4), columns = pca_results.index.values))

### Implementation: Dimensionality Reduction
When using principal component analysis, one of the main goals is to reduce the dimensionality of the data — in effect, reducing the complexity of the problem. Dimensionality reduction comes at a cost: Fewer dimensions used implies less of the total variance in the data is being explained. Because of this, the *cumulative explained variance ratio* is extremely important for knowing how many dimensions are necessary for the problem. Additionally, if a signifiant amount of variance is explained by only two or three dimensions, the reduced data can be visualized afterwards.

In the code block below, you will need to implement the following:
 - Assign the results of fitting PCA in two dimensions with `good_data` to `pca`.
 - Apply a PCA transformation of `good_data` using `pca.transform`, and assign the reuslts to `reduced_data`.
 - Apply a PCA transformation of the sample log-data `log_samples` using `pca.transform`, and assign the results to `pca_samples`.

In [ ]:
# TODO: Fit PCA to the good data using only two dimensions
pca = PCA(n_components=2).fit(good_data)

# TODO: Apply a PCA transformation the good data
reduced_data = pca.transform(good_data)

# TODO: Apply a PCA transformation to the sample log-data
pca_samples = pca.transform(log_samples)

# Create a DataFrame for the reduced data
reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2'])

### Observation
Run the code below to see how the log-transformed sample data has changed after having a PCA transformation applied to it using only two dimensions. Observe how the values for the first two dimensions remains unchanged when compared to a PCA transformation in six dimensions.

In [ ]:
# Display sample log-data after applying PCA transformation in two dimensions
display(pd.DataFrame(np.round(pca_samples, 4), columns = ['Dimension 1', 'Dimension 2']))

## Clustering

In this section, you will choose to use either a K-Means clustering algorithm or a Gaussian Mixture Model clustering algorithm to identify the various customer segments hidden in the data. You will then recover specific data points from the clusters to understand their significance by transforming them back into their original dimension and scale. 

### Question 6
*What are the advantages to using a K-Means clustering algorithm? What are the advantages to using a Gaussian Mixture Model clustering algorithm? Given your observations about the wholesale customer data so far, which of the two algorithms will you use and why?*

**Answer:**
The advantages to using K-Means clustering algorithm are that K-Means clustering may be computationally faster, more robust and easier to understand than hierarchical clustering (if K is small). K-Means clustering, gives best result when data points are distinct or well seperate from each other, may produce tighter clusters than hierarchical clustering, especially if clusters are globular.

The advatages of Gaussian Mixture Model clustering is that it can accommodate clusters that have different sizes and correlation structures within them. As this algorithm maximizes only the likelihood, it will not bias the means towards zero or bias the cluster sizes to have specific structures that might or might not apply.

Given my observations about the wholesale customer data so far, I will rather use K-Means clustering algorithm because it is simple, fast, scalable, doesn't assign any given point to two different clusters with different scores and most importantly works well with large number of data.

references: 
http://www.improvedoutcomes.com/docs/WebSiteDocs/Clustering/K-Means_Clustering_Overview.htm
http://www.mathworks.com/help/stats/clustering-using-gaussian-mixture-models.html



### Implementation: Creating Clusters
Depending on the problem, the number of clusters that you expect to be in the data may already be known. When the number of clusters is not known *a priori*, there is no guarantee that a given number of clusters best segments the data, since it is unclear what structure exists in the data — if any. However, we can quantify the "goodness" of a clustering by calculating each data point's *silhouette coefficient*. The [silhouette coefficient](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) for a data point measures how similar it is to its assigned cluster from -1 (dissimilar) to 1 (similar). Calculating the *mean* silhouette coefficient provides for a simple scoring method of a given clustering.

In the code block below, you will need to implement the following:
 - Fit a clustering algorithm to the `reduced_data` and assign it to `clusterer`.
 - Predict the cluster for each data point in `reduced_data` using `clusterer.predict` and assign them to `preds`.
 - Find the cluster centers using the algorithm's respective attribute and assign them to `centers`.
 - Predict the cluster for each sample data point in `pca_samples` and assign them `sample_preds`.
 - Import sklearn.metrics.silhouette_score and calculate the silhouette score of `reduced_data` against `preds`.
   - Assign the silhouette score to `score` and print the result.

In [ ]:
from sklearn.cluster import KMeans

# TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
from sklearn.metrics import silhouette_score

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters = n_clusters)
    preds = clusterer.fit_predict(reduced_data)
    score = silhouette_score(reduced_data, preds)
    print "For number of clusters = {}, the silhouette score is : {:.6f}".format(n_clusters, score)

# TODO: Apply your clustering algorithm of choice to the reduced data
clusterer = KMeans(n_clusters=2)

# TODO: Predict the cluster for each data point
preds = clusterer.fit_predict(reduced_data)

# TODO: Find the cluster centers
centers = clusterer.cluster_centers_

# TODO: Predict the cluster for each transformed sample data point
sample_preds = clusterer.predict(pca_samples)



### Question 7
*Report the silhouette score for several cluster numbers you tried. Of these, which number of clusters has the best silhouette score?* 

**Answer:**
The number of clusters = 2 has the best silhouette score of approximately 0.426281

### Cluster Visualization
Once you've chosen the optimal number of clusters for your clustering algorithm using the scoring metric above, you can now visualize the results by executing the code block below. Note that, for experimentation purposes, you are welcome to adjust the number of clusters for your clustering algorithm to see various visualizations. The final visualization provided should, however, correspond with the optimal number of clusters. 

In [ ]:
# Display the results of the clustering from implementation
rs.cluster_results(reduced_data, preds, centers, pca_samples)

### Implementation: Data Recovery
Each cluster present in the visualization above has a central point. These centers (or means) are not specifically data points from the data, but rather the *averages* of all the data points predicted in the respective clusters. For the problem of creating customer segments, a cluster's center point corresponds to *the average customer of that segment*. Since the data is currently reduced in dimension and scaled by a logarithm, we can recover the representative customer spending from these data points by applying the inverse transformations.

In the code block below, you will need to implement the following:
 - Apply the inverse transform to `centers` using `pca.inverse_transform` and assign the new centers to `log_centers`.
 - Apply the inverse function of `np.log` to `log_centers` using `np.exp` and assign the true centers to `true_centers`.


In [ ]:
# TODO: Inverse transform the centers
log_centers = pca.inverse_transform(centers)

# TODO: Exponentiate the centers
true_centers = np.exp(log_centers)

# Display the true centers
segments = ['Segment {}'.format(i) for i in range(0,len(centers))]
true_centers = pd.DataFrame(np.round(true_centers), columns = data.keys())
true_centers.index = segments
display(true_centers)

### Question 8
Consider the total purchase cost of each product category for the representative data points above, and reference the statistical description of the dataset at the beginning of this project. *What set of establishments could each of the customer segments represent?*  
**Hint:** A customer who is assigned to `'Cluster X'` should best identify with the establishments represented by the feature set of `'Segment X'`.

In [ ]:
true_centers = true_centers.append(data.describe().loc['mean'])
my_centers = true_centers.plot(kind='bar', figsize=(15,6) )

**Answer:**
Segment 0 is in cluster 0 and segment 1 belong to cluster 1
Segment 0 has a signifiacant number of Grocery well above the mean for all categories(about 1 and half time the mean) and miningful number of Milk(about 1.4 the mean) and also Detergents_Paper(about 1.4 the mean) were purchased. This Segment could represent retailers because retailers normally sell lots of Grocery, milk, and Detergents_Paper which are shown to be purchased higher than mean from this whosale distribution.
Segment 1 purchased less than average for all categories, which is understandable because people go to or buy for example more Grocery or Detergents_Paper from retail stores than cafe, Hotel, or a Restaurant. However, Segment 1 selling power is shown for the purchase of Fresh and Frozen for which they purchased more than Segment 0(retailers), meaning that Cafe, Hotel, and/or Restaurants sell more of  those two products.

### Question 9
*For each sample point, which customer segment from* ***Question 8*** *best represents it? Are the predictions for each sample point consistent with this?*

Run the code block below to find which cluster each sample point is predicted to be.

In [ ]:
# Display the predictions
for i, pred in enumerate(sample_preds):
    print "Sample point", i, "predicted to be in Cluster", pred

**Answer:**
The predictions for sample points are consistent with the corresponding segments. Because from the results of the clustering plot, sample point 0, marked with black cross is located closer to center 0 cluster(retailers). As I mentioned earlier in question 8, retailers could represent segment 0(center 0 cluster) and Restaurant, Hotels, and cafe have similar spending habits in relation to retailers. Sample points 1 and sample point 2 are both closer to center 1 cluster which could be either Restaurant, Hotel or cafe. Comparing both samples in the same cluster(cluster 1), Sample point 1 could represent a Restaurant because it purchased more Frozen and Fresh than sample point 1. Which is in some respects, some Hotels typical spending habits. For sample point 2, the values for Milk, Grocery, Detrgents_Paper are little higher than those for sample point 1, I would argue that this point represents a restaurant.

## Conclusion

### Question 10
*Companies often run [A/B tests](https://en.wikipedia.org/wiki/A/B_testing) when making small changes to their products or services. If the wholesale distributor wanted to change its delivery service from 5 days a week to 3 days a week, how would you use the structure of the data to help them decide on a group of customers to test?*  
**Hint:** Would such a change in the delivery service affect all customers equally? How could the distributor identify who it affects the most?

**Answer:**
I would consider 5 days a week delivery service to be variant A and 3 days a week delivery service to be variant B. I would then evaluate how much product get purchased by each customer in variant A versus Variant B. Depending on the evaluation I would consider whether:
To change the delivery service from 5 days to 3 days a week for all customers if they all purchased more products in variant B than Variant A.
To keep 5 days a week delivery service for all customers if they all purchased less products in variant B than Variant A.

To classify customers in to two groups, those who purchased more products in variant A and those who purchased more products in variant B. In this case I would offer 5 days a week delivery service to the group that purchased more products in variant A and 3 days a week delivery service to the one that purchased more products in variant B to maximize the purchasing power and minimize the cost of delivery service.

To find out whether the change from 5 days week to 3 days delivery service a week affect all customers equally. I would test the 3 days a week delivery service on cluster 0 and cluster 1 of the wholesale distribution data. if the same number of customers from both clusters' subsets purchased less or the same amount of products as before then I would agree that it affects them equally, otherwise I would see whether cluster 0 or cluster 1 had less number of customers' purchases as before. The cluster that would have less number of customers with the modified delivery service would be identified as the most affected by the new delivery system.     

### Question 11
*Assume the wholesale distributor wanted to predict a new feature for each customer based on the purchasing information available. How could the wholesale distributor use the structure of the data to assist a supervised learning analysis?*  
**Hint:** What other input feature could the supervised learner use besides the six product features to help make a prediction?

**Answer:**
 Based on the purchasing information available, customers are divided into two segments or two channels. To assist the supervised learning analysis, the distributor uses the structure of the data by assigning 0 and 1 as value of the new feature. 

### Visualizing Underlying Distributions

At the beginning of this project, it was discussed that the `'Channel'` and `'Region'` features would be excluded from the dataset so that the customer product categories were emphasized in the analysis. By reintroducing the `'Channel'` feature to the dataset, an interesting structure emerges when considering the same PCA dimensionality reduction applied earlier on to the original dataset.

Run the code block below to see how each data point is labeled either `'HoReCa'` (Hotel/Restaurant/Cafe) or `'Retail'` the reduced space. In addition, you will find the sample points are circled in the plot, which will identify their labeling.

In [ ]:
# Display the clustering results based on 'Channel' data
rs.channel_results(reduced_data, outliers, pca_samples)

### Question 12
*How well does the clustering algorithm and number of clusters you've chosen compare to this underlying distribution of Hotel/Restaurant/Cafe customers to Retailer customers? Are there customer segments that would be classified as purely 'Retailers' or 'Hotels/Restaurants/Cafes' by this distribution? Would you consider these classifications as consistent with your previous definition of the customer segments?*

**Answer:**
The clustering algorithm and the number of clusters I've chosen compared to this underlying distribution of Hotel/Restaurant/Cafe customers to Retailer customers are well determined with some level of impurity. I can see around both clusters, few blue small cirled data points labeled Hotel/Restaurant/Cafe among the red small circles area, in another word some Hotel/Restaurant/Cafe points are mixed with or very close to retailer points and similarly, few red cirles are mixed in the blue ones from the above display as well as the previous plots. Therefore, there are no customers that would be classified as purely 'Retailer' or 'Hotels/Restaurants/Cafes'. Yes I would also consider these classifications as consistent with my previous definition of the customer segments. This is because I would say that the amount a given customer could purchase from its distributor can vary depending on its stock or the amount sold.


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.